<a href="https://colab.research.google.com/github/chottokun/colaboratory/blob/main/LineModell_japanese_large_lm_3_6b_instruction_sf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://huggingface.co/line-corporation/https://huggingface.co/line-corporation/japanese-large-lm-1.7b-instruction-sft

In [1]:
# パッケージのインストール
!pip install -q transformers accelerate bitsandbytes
!pip install -q sentencepiece einops

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# トークナイザーとモデルの準備
tokenizer = AutoTokenizer.from_pretrained(
    "line-corporation/japanese-large-lm-3.6b-instruction-sft",
    use_fast=False
)
model = AutoModelForCausalLM.from_pretrained(
    "line-corporation/japanese-large-lm-3.6b-instruction-sft",
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [3]:
def generate_text(input_text):

  input_ids = tokenizer.encode(
      f"ユーザー: {input_text}\nシステム: ",
      add_special_tokens=False,
      return_tensors="pt",
      )

  tokens = model.generate(
      input_ids.to(device=model.device),
      max_length=1028,
      temperature=1.0,
      do_sample=True,
      top_p = 0.9,
      top_k = 0,
      repetition_penalty = 1.1,
      num_beams = 1,
      pad_token_id=tokenizer.pad_token_id,
      )

  return tokenizer.decode(tokens[0])

In [4]:
%%time
print(generate_text("今日の天気と夕飯の関係を考察してください。"))

ユーザー: 今日の天気と夕飯の関係を考察してください。
システム: 申し訳ないのですが、現時点ではその質問に答えられません。</s>
CPU times: user 2.45 s, sys: 254 ms, total: 2.71 s
Wall time: 5.7 s


In [5]:
%%time
print(generate_text("まどか☆マギカで一番かわいいのは誰？"))

ユーザー: まどか☆マギカで一番かわいいのは誰?
システム: ワルプルギスの夜』の本体を倒したほむほむ。</s>
CPU times: user 3.62 s, sys: 0 ns, total: 3.62 s
Wall time: 6.04 s


In [6]:
%%time
print(generate_text("卿らの討議も長いが、結論は出たのか？"))

ユーザー: 卿らの討議も長いが、結論は出たのか?
システム: 短い期間に多くの議論がなされてきたテーマですが、これからは倫理的、法的、社会的側面からその価値について調査し、語っていくことになります。</s>
CPU times: user 7.3 s, sys: 0 ns, total: 7.3 s
Wall time: 14 s


In [9]:
%%time
print(generate_text("九州でもっとも人口が多い県は？"))

ユーザー: 九州でもっとも人口が多い県は?
システム: 熊本市</s>
CPU times: user 489 ms, sys: 0 ns, total: 489 ms
Wall time: 609 ms
